In [1]:
import json
from collections import defaultdict
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv("../Data/Interim/some_cmd_v3_03062022_low_split.csv", sep=",")
df = df.loc[:,["full_text", "text1", "text2", "text3", "subset"]]
print(df.shape)
# добавляем кома
df_onecmd = pd.read_csv("../Data/Interim/one_cmd_v3_03062022_low_split.csv", sep=",")
df_onecmd = df_onecmd.sample(5000, random_state=42)
df_onecmd = df_onecmd.loc[df_onecmd["subset"].isin(["train", "valid"]), ["x", "subset"]]
df_onecmd["full_text"] = df_onecmd["x"]
df_onecmd["text1"] = df_onecmd["x"]
df_onecmd["text2"] = np.empty_like(df_onecmd["x"], dtype=np.float32)
df_onecmd["text3"] = np.empty_like(df_onecmd["x"], dtype=np.float32)

df = pd.concat([df, df_onecmd])
df.index= range(len(df))

(16253, 5)


In [3]:
nerInput_train = {
    "sentence_id": [],
    "words": [],
    "labels": []
}
nerInput_valid = {
    "sentence_id": [],
    "words": [],
    "labels": []
}
nerInput = None
for ir, row in df.iterrows():
    if row["subset"]=="train":
        nerInput = nerInput_train
    if row["subset"]=="valid":
        nerInput = nerInput_valid

    start_1 = row["full_text"].find(row["text1"])
    assert start_1 >= 0
    start_2 = None
    if type(row['text2']) != float:
        start_2 = row["full_text"].find(row["text2"], start_1 + len(row["text1"]))    
        assert start_2 >= start_1
    start_3 = None
    if type(row['text3']) != float:
        start_3 = row["full_text"].find(row["text3"], start_2 + len(row["text2"]))
        assert start_3 >= start_2
    lastWordPos = 0
    for word in row["full_text"].split():
        nerInput["sentence_id"].append(ir)
        nerInput["words"].append(word)
        wordPos = row["full_text"].find(word, lastWordPos)
        assert wordPos!=-1
        lastWordPos = wordPos + len(word)
        if wordPos >= start_1 and wordPos < (start_1+len(row["text1"])):
            nerInput["labels"].append("CMD1")
        elif (start_2 is not None) and (wordPos >= start_2) and (wordPos < (start_2+len(row["text2"]))):
            nerInput["labels"].append("CMD2")
        elif (start_3 is not None) and (wordPos >= start_3) and (wordPos < (start_3+len(row["text3"]))):
            nerInput["labels"].append("CMD3")
        else:
            nerInput["labels"].append("O")
train_df = pd.DataFrame(nerInput_train)
valid_df = pd.DataFrame(nerInput_valid)

In [4]:
ir = 5000
print(df.loc[ir, "full_text"])
ner_commands= [[],[],[]]
for ir_2, row in train_df[train_df["sentence_id"]==ir].iterrows():
    if row["labels"]=="CMD1":
        ner_commands[0].append(row["words"])
    if row["labels"]=="CMD2":
        ner_commands[1].append(row["words"])
    if row["labels"]=="CMD3":
        ner_commands[2].append(row["words"])
print(df.loc[ir, "text1"], "|", " ".join(ner_commands[0]))
print(df.loc[ir, "text2"], "|", " ".join(ner_commands[1]))
print(df.loc[ir, "text3"], "|", " ".join(ner_commands[2]))

отвали сюда немного спустя оставь позади ближайший валун спустя время поворачивай 27 °
отвали сюда | отвали сюда
оставь позади ближайший валун | оставь позади ближайший валун
поворачивай 27 ° | поворачивай 27 °


In [5]:
from simpletransformers.ner import NERModel, NERArgs
model_args = NERArgs()
model_args.evaluate_during_training = True
model_args.num_train_epochs = 10
model_args.output_dir = "/s/ls4/users/grartem/RL_robots/CommandClassifier/models/TokenCLS_splitter_ipynb_datav3v2"

model = NERModel(
    "bert", 
    "/s/ls4/users/grartem/HuggingFace/models/rubert-tiny2",
    #"/s/ls4/users/grartem/RL_robots/CommandClassifier/models/Preprocess_for_splitter_ntbk/checkpoint-18500-epoch-10/",
    args=model_args,
    labels=["O", "CMD1", "CMD2", "CMD3"] # добавить тэг O, но по границам
)

Some weights of the model checkpoint at /s/ls4/users/grartem/HuggingFace/models/rubert-tiny2 were not used when initializing BertForTokenClassification: ['cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassific

In [6]:
model_args.output_dir

'/s/ls4/users/grartem/RL_robots/CommandClassifier/models/TokenCLS_splitter_ipynb_datav3'

In [7]:
# Train the model
model.train_model(train_df, eval_data=valid_df)

  0%|          | 0/29 [00:00<?, ?it/s]

/s/ls4/users/grartem/anaconda3/envs/simptr/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 0 of 10:   0%|          | 0/2264 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

...

/s/ls4/users/grartem/anaconda3/envs/simptr/lib/python3.8/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: CMD1 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/s/ls4/users/grartem/anaconda3/envs/simptr/lib/python3.8/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: CMD2 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/s/ls4/users/grartem/anaconda3/envs/simptr/lib/python3.8/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: CMD3 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


  0%|          | 0/7 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/380 [00:00<?, ?it/s]

/s/ls4/users/grartem/anaconda3/envs/simptr/lib/python3.8/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: CMD1 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/s/ls4/users/grartem/anaconda3/envs/simptr/lib/python3.8/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: CMD2 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/s/ls4/users/grartem/anaconda3/envs/simptr/lib/python3.8/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: CMD3 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


(22640,
 defaultdict(list,
             {'global_step': [2000,
               2264,
               4000,
               4528,
               6000,
               6792,
               8000,
               9056,
               10000,
               11320,
               12000,
               13584,
               14000,
               15848,
               16000,
               18000,
               18112,
               20000,
               20376,
               22000,
               22640],
              'train_loss': [0.036302607506513596,
               0.0015373582718893886,
               0.00032926740823313594,
               0.00020968914031982422,
               0.00015315745258703828,
               7.82379720476456e-05,
               9.953257540473714e-05,
               3.980673136538826e-05,
               2.699717879295349e-05,
               3.741847103810869e-05,
               2.3814764062990434e-05,
               0.0005106310709379613,
               4.89221019961405

In [8]:
# Evaluate the model
result, model_outputs, preds_list = model.eval_model(valid_df)

  0%|          | 0/7 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/380 [00:00<?, ?it/s]

/s/ls4/users/grartem/anaconda3/envs/simptr/lib/python3.8/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: CMD1 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/s/ls4/users/grartem/anaconda3/envs/simptr/lib/python3.8/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: CMD2 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/s/ls4/users/grartem/anaconda3/envs/simptr/lib/python3.8/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: CMD3 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


In [9]:
predictions, raw_outputs = model.predict(df[df["subset"]=="valid"]["full_text"])

  0%|          | 0/7 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/380 [00:00<?, ?it/s]

# оценка

In [12]:
from sklearn.metrics import classification_report
flat_token_predictions = [list(x.values())[0] for sent in predictions for x in sent]
flat_token_true = valid_df["labels"].values
print(classification_report(flat_token_true, flat_token_predictions))

              precision    recall  f1-score   support

        CMD1       1.00      1.00      1.00     15128
        CMD2       1.00      1.00      1.00     10950
        CMD3       1.00      1.00      1.00      3064
           O       1.00      1.00      1.00      2959

    accuracy                           1.00     32101
   macro avg       1.00      1.00      1.00     32101
weighted avg       1.00      1.00      1.00     32101



In [13]:
i, errors = 0, 0
for ir, row in df[df["subset"]=="valid"].iterrows():
    text1, text2, text3 = [], [], []
    for token in predictions[i]:
        token, cls = list(token.items())[0]
        if cls=="CMD1":
            text1.append(token)
        if cls=="CMD2":
            text2.append(token)
        if cls=="CMD3":
            text3.append(token)
    text1 = " ".join(text1)
    text2 = " ".join(text2)
    text3 = " ".join(text3)
    
    try:
        assert text1 == row["text1"].strip()
        assert (text2=="" and type(row["text2"]) is float) or text2 == row["text2"].strip()
        assert (text3=="" and type(row["text3"]) is float) or text3 == row["text3"].strip()
        #assert (text3=="" and np.isnan(row["text3"])) or text3 == row["text3"].strip()
    except AssertionError:
        print("full_text:", row["full_text"])
        print("text1:", row["text1"], "predict:", text1)
        print("text2:", row["text2"], "predict:", text2)
        print("text3:", row["text3"], "predict:", text3)
        print()
        errors += 1
    i+=1
print("Errors", errors)

full_text: приедь к авто левее сзади от чащи находится холмик который надо обставить позже поворачивай до возвышенности правее себя
text1: приедь к авто левее  predict: приедь к авто
text2: сзади от чащи находится холмик который надо обставить predict: левее сзади от чащи находится холмик который надо обставить
text3: поворачивай до возвышенности правее себя predict: поворачивай до возвышенности правее себя

full_text: сделай поворот до канавы передохни с течением времени приспевай назад
text1: сделай поворот до канавы predict: сделай поворот до канавы передохни
text2: передохни predict: 
text3: приспевай назад predict: приспевай назад

full_text: начни поворачивать 343° немного погодя стоять правее дерева располагается тачка к которой приказывается плыть
text1: начни поворачивать 343° predict: начни поворачивать 343°
text2: стоять predict: стоять правее дерева располагается тачка к которой приказывается плыть
text3: правее дерева располагается тачка к которой приказывается плыть predi

In [14]:
df = pd.read_csv("../Data/Interim/one_cmd_v3_03062022_low_split.csv", sep=",")
predictions, raw_outputs = model.predict(df["x"])

  0%|          | 0/29 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/7136 [00:00<?, ?it/s]

In [17]:
errors_count = 0
for phrase in predictions:
    classes = [list(x.values())[0] for x in phrase]
    if len(set(classes))==1 and classes[0]=="CMD1":
        continue
    else:
        errors_count+=1
errors_count, len(predictions)

(330, 57085)